In [1]:
import pandas as pd

filename = '61600ds0001_2015-16.xls'

sheet1 = pd.read_excel(filename,sheet_name=1,header=None)
sheet1.to_csv('abs_sheet_1.csv',index = False)

coords = [(6,21),(27,20),(47,6),(53,4),(57,5),(62,5),(67,1)]
path = 'abs_sheet_1.csv'

chunks = [pd.read_csv(path, skiprows = x, header = None, nrows = y) for x,y in coords]

cols0 = [c for c in chunks[0].iloc[0,:] if pd.notnull(c)]
cols1 = pd.unique([c for c in chunks[0].iloc[1,:] if pd.notnull(c)])
cols2 = pd.unique([c for c in chunks[0].iloc[2,:] if pd.notnull(c)])

chunk_data = [chunk.drop(0).drop(columns = [0,1]) for chunk in chunks[1:-1]]
chunk_data.append(chunks[-1].drop(columns = [0,1]))
chunk_data.insert(0,chunks[0].drop([0,1,2,3]).drop(columns = [0,1]))

cols_index = pd.MultiIndex.from_product([cols0,cols1,cols2])

for c in chunk_data: c.columns = cols_index

new_index_list = []
for c in chunks[1:-1]:
    data = c[0].tolist()
    index_0, index_1 = [data.pop(0)],data
    new_index = pd.MultiIndex.from_product([index_0, index_1])
    new_index_list.append(new_index)

for k,v in zip(new_index_list,chunk_data[1:-1]):
    v.index = k

data = chunks[0].drop([0,1,2])[0].tolist()
index_0, index_1 = [data.pop(0)],data
new_index = pd.MultiIndex.from_product([index_0, index_1])
chunk_data[0].index = new_index

chunk_data[-1].index = pd.MultiIndex.from_product([chunks[-1][0],chunks[-1][0]])

In [2]:
full_df = chunk_data[0].append(chunk_data[1:])

full_df.to_pickle('full_dataset.p')

names = ['age_group', 'industry', 
         'institutional_sector', 'type_of_legal_organisation',
         'employment_size', 'job_duration', 'total']

for k,v in zip(names, chunk_data):
    pd.to_pickle(v, k + '.p')